In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Resistance data

In [2]:
resistance_data = pd.read_csv('antibio_dataset/resist.csv')

In [4]:
resistance_data.shape

(3786, 4)

Missing values per column:

In [6]:
resistance_data.isna().sum()

Sample_ID      0
azm_sr       308
cip_sr       698
cfx_sr       385
dtype: int64

Proportion of missing values per column:

In [8]:
resistance_data.isna().sum() / len(resistance_data)

Sample_ID    0.000000
azm_sr       0.081352
cip_sr       0.184363
cfx_sr       0.101690
dtype: float64

Although the share of missing values per column is not significantly high, one would still aim to keep as much data points as possible for the prediction analysis. Thus imputation of missing values would be a possible step but in this case the authors hold no information about the biological similarity of the samples. Not being certain if Sample_ID string similarity would also translate into biological dimilarity between the samples, authors decided not to impute the missing values without additional similarity measures and omit the samples that have missing values from the analysis entirely.

In [16]:
azithromycin_res = resistance_data[resistance_data['azm_sr'].isna() != True][['Sample_ID','azm_sr']]
cefixime_res = resistance_data[resistance_data['cfx_sr'].isna() != True][['Sample_ID','cfx_sr']]
ciprofloxacin_res = resistance_data[resistance_data['cip_sr'].isna() != True][['Sample_ID','cip_sr']]

In [21]:
print(f"Azithromycin: {azithromycin_res.shape[0]} samples")
print(f"Cefixime: {cefixime_res.shape[0]} samples")
print(f"Ciprofloxacin: {ciprofloxacin_res.shape[0]} samples")

Azithromycin: 3478 samples
Cefixime: 3401 samples
Ciprofloxacin: 3088 samples


### K-mer data

In [61]:
kmers_azithromycin = pd.read_csv('antibio_dataset/kmers_azithromycin.tsv', sep="\t")
kmers_cefixime = pd.read_csv('antibio_dataset/kmers_cefixime.tsv', sep="\t")
kmers_ciprofloxacin = pd.read_csv('antibio_dataset/kmers_ciprofloxacin.tsv', sep="\t")

In [62]:
kmers_azithromycin.head()

,pattern_id,ERR1549286,ERR1549290,ERR1549291,ERR1549287,ERR1549288,ERR1549299,ERR1549292,ERR1549298,ERR1549296,...,ERR2172345,ERR2172346,ERR2172347,ERR2172348,ERR2172349,ERR2172350,ERR2172351,ERR2172352,ERR2172353,ERR2172354
0,CTTAACATATTTGCCTTTGATTTTTGAAGAAGCTGCCACGCCGGCAG,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,TACCGTAACCGGCAATGCGGATATTACGGTC,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,CAGACGGCATTTTTTTTGCGTTTTTCGGGAGG,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,AACGGGTTTTCAGACGGCATTCGATATCGGGACG,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,CCAAAAATTACCCGCGTTGACGTAGCTAAAGA,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In order to merge the information of the resistance data tables and k-mer data tables, the k-mer tables need to be transposed (sample IDs in rows and patterns in columns).

#### Azithromycin

In [63]:
kmers_azithromycin = kmers_azithromycin.transpose()

#Making the first row of the transposed DataFrame as column headers
new_header = kmers_azithromycin.iloc[0]
kmers_azithromycin = kmers_azithromycin[1:] 
kmers_azithromycin.columns = new_header

#Making the structure of column names similar to the structure of resistance tables
kmers_azithromycin = kmers_azithromycin.reset_index()
kmers_azithromycin = kmers_azithromycin.rename(columns={'index': 'Sample_ID'})
kmers_azithromycin.index.name = None

#### Cefixime

In [66]:
kmers_cefixime = kmers_cefixime.transpose()

#Making the first row of the transposed DataFrame as column headers
new_header = kmers_cefixime.iloc[0]
kmers_cefixime = kmers_cefixime[1:] 
kmers_cefixime.columns = new_header

#Making the structure of column names similar to the structure of resistance tables
kmers_cefixime = kmers_cefixime.reset_index()
kmers_cefixime = kmers_cefixime.rename(columns={'index': 'Sample_ID'})
kmers_cefixime.index.name = None

#### Ciprofloxacin

In [68]:
kmers_ciprofloxacin = kmers_ciprofloxacin.transpose()

#Making the first row of the transposed DataFrame as column headers
new_header = kmers_ciprofloxacin.iloc[0]
kmers_ciprofloxacin = kmers_ciprofloxacin[1:] 
kmers_ciprofloxacin.columns = new_header

#Making the structure of column names similar to the structure of resistance tables
kmers_ciprofloxacin = kmers_ciprofloxacin.reset_index()
kmers_ciprofloxacin = kmers_ciprofloxacin.rename(columns={'index': 'Sample_ID'})
kmers_ciprofloxacin.index.name = None

### Merging the data

In [72]:
azithromycin_data = kmers_azithromycin.merge(azithromycin_res, how = 'right', on = 'Sample_ID')
cefixime_data = kmers_cefixime.merge(cefixime_res, how = 'right', on = 'Sample_ID')
ciprofloxacin_data = kmers_ciprofloxacin.merge(ciprofloxacin_res, how = 'right', on = 'Sample_ID')